In [1]:
import sys, numpy, pandas, polars, matplotlib, seaborn, sklearn; print(f"✅ Python {sys.version.split()[0]} | NumPy {numpy.__version__} | Pandas {pandas.__version__} | Polars {polars.__version__} | Matplotlib {matplotlib.__version__} | Seaborn {seaborn.__version__} | Scikit-learn {sklearn.__version__}")

Matplotlib is building the font cache; this may take a moment.


✅ Python 3.13.3 | NumPy 2.3.2 | Pandas 2.3.1 | Polars 1.31.0 | Matplotlib 3.10.3 | Seaborn 0.13.2 | Scikit-learn 1.7.1
